In [1]:
import contract_utilities.expiration as exp
date_list = exp.get_bus_day_list(date_from=20161101, date_to=20170217)
date_list

backtest_list = []

import opportunity_constructs.technical_scalper as ts

for i in range(len(date_list)):
    print(date_list[i])
    backtest_list.append(ts.get_ts_results_4date(date_to=date_list[i]))

20161101
20161102
20161103
20161104
20161107
20161108
20161109
20161110
20161111
20161114
20161115
20161116
20161117
20161118
20161121
20161122
20161123
20161125
20161128
20161129
20161130
20161201
20161202
20161205
20161206
20161207
20161208
20161209
20161212
20161213
20161214
20161215
20161216
20161219
20161220
20161221
20161222
20161223
20161227
20161228
20161229
20161230
20170103
20170104
20170105
20170106
20170109
20170110
20170111
20170112
20170113
20170117
20170118
20170119
20170120
20170123
20170124
20170125
20170126
20170127
20170130
20170131
20170201
20170202
20170203
20170206
20170207
20170208
20170209
20170210
20170213
20170214
20170215
20170216
20170217


In [96]:
# Cross Sectional Opportunities?

pnl_frame = pd.concat(backtest_list)
import numpy as np

long_pnl_list = []
short_pnl_list = []
settle_date_list = []


unique_dates = pnl_frame['settle_date'].unique()

for i in range(len(unique_dates)):
    
    daily_data = pnl_frame[pnl_frame['settle_date']==unique_dates[i]]
    unique_hour_minute = daily_data['hour_minute'].unique()
    
    for j in range(len(unique_hour_minute)):
        hour_minute_data = daily_data[daily_data['hour_minute']==unique_hour_minute[j]]
        hour_minute_data = hour_minute_data[hour_minute_data['hourly_normalized_volume']<0.25]

        hour_minute_data.sort('ma50_spread',ascending=True,inplace=True)
        long_pnl_list.append(np.mean(hour_minute_data['PerContractPnl60'].iloc[:1]))
        short_pnl_list.append(-np.mean(hour_minute_data['PerContractPnl60'].iloc[-1:]))
        settle_date_list.append(unique_dates[i])
        
cross_frame = pd.DataFrame.from_items([('long', long_pnl_list),('short', short_pnl_list),
                                       ('settle_date',settle_date_list)])
cross_frame['total'] = cross_frame['long'] + cross_frame['short'] 

In [101]:
net_winner = cross_frame[cross_frame['total']>=0]
net_looser = cross_frame[cross_frame['total']<0]

len(net_winner)/(len(net_looser)+len(net_looser))
print(net_winner['total'].mean())
print(net_looser['total'].mean())
print(cross_frame['total'].mean())

238.36163321167876
-221.79616310160435
26.52138724766129


In [95]:
grouped = cross_frame.groupby('settle_date')
daily = pd.DataFrame()
daily['total'] = grouped['total'].mean()
16*daily['total'].mean()/daily['total'].std()

5.0559233312667144

In [3]:
import shared.statistics as stats
pnl_frame = pd.concat(backtest_list)
quantile_output = stats.get_number_from_quantile(y=select_frame['hourly_normalized_volume'].values,quantile_list=[25,50,75])
quantile_output

array([ 0.06642251,  0.10289273,  0.14841708])

In [8]:


import backtesting.backtest_stats as bs
import pandas as pd

import shared.statistics as stats
pnl_frame = pd.concat(backtest_list)
select_frame = pnl_frame[pnl_frame['hourly_normalized_volume']<0.25]
#select_frame = pnl_frame
bs.get_indicator_rr_table(trade_data=select_frame,indicator_name='ma50_spread',
                                 strategy_class='ts',
                                 long_pnl_field='NormPnl60',
                                            short_pnl_field='NormPnl60_')

,indicator_ulimit,mean_pnl,reward_risk
0,-0.384720,84.811795,0.077022
1,-0.221603,-48.692938,-0.048970
2,-0.115676,-10.114950,-0.011764
3,-0.031980,12.951190,0.015819
4,0.044825,-32.667797,-0.038631
5,0.127757,40.731283,0.050296
6,0.232735,25.244220,0.030546
7,0.391320,8.472347,0.009342
8,NaN,135.265855,0.131106


In [3]:
import pandas as pd
import shared.statistics as stats
pnl_frame = pd.concat(backtest_list)

indicator_name = 'NormCloseChange_15'


value_list = stats.get_number_from_quantile(y = pnl_frame[indicator_name].values,quantile_list=[5,95])
small_frame = pnl_frame[pnl_frame[indicator_name]<value_list[0]]
big_frame = pnl_frame[pnl_frame[indicator_name]>value_list[1]]

print(small_frame['NormPnl60'].mean()/small_frame['NormPnl60'].std())
print(big_frame['NormPnl60'].mean()/big_frame['NormPnl60'].std())

# ewma10_spread 0.05 total contrarian spread
# ewma20_spread 0.028 total contrarian spread
# ewma50_spread 0.03 total contrarian spread
# ewma100_spread 0.02 total contratian spread




0.00034228560968566394
-0.049910143646792866


In [2]:
import backtesting.backtest_stats as bs
import pandas as pd
from IPython.display import display

import pandas as pd
pnl_frame = pd.concat(backtest_list)

pnl_frame = pnl_frame[pnl_frame['hourly_normalized_volume']<0.25]

rank_indicators_output = bs.rank_indicators(trade_data=pnl_frame,
                                            indicator_list = ['ma10_spread','ma20_spread','ma50_spread',
                                                              'ma100_spread','ts_slope5', 'ts_slope10', 'ts_slope20',
                                                              'linear_deviation5', 'linear_deviation10','linear_deviation20',
                                                              'NormCloseChange_15','NormCloseChange_60'],
                                            long_pnl_field='NormPnl60',
                                            short_pnl_field='NormPnl60_',
                                            strategy_class='ts',granular_ranking_type='ticker_head')
indicator_ranking_total = rank_indicators_output['indicator_ranking_total']
indicator_ranking_granular_total = rank_indicators_output['indicator_ranking_granular_total']
display(indicator_ranking_total)
display(indicator_ranking_granular_total)

,indicator,ranking
80,"[ts_slope20, ma50_spread]",280
47,"[ma100_spread, ma50_spread]",278
81,"[ts_slope20, ma100_spread]",277
2,ma50_spread,275
24,"[ma20_spread, ma50_spread]",260
50,"[ma100_spread, ts_slope20]",260
3,ma100_spread,259
69,"[ts_slope10, ma50_spread]",257
135,"[NormCloseChange_60, ma50_spread]",257
39,"[ma50_spread, ts_slope20]",255


,indicator,ranking
80,"[ts_slope20, ma50_spread]",5908
2,ma50_spread,5898
47,"[ma100_spread, ma50_spread]",5853
24,"[ma20_spread, ma50_spread]",5720
135,"[NormCloseChange_60, ma50_spread]",5697
35,"[ma50_spread, ma20_spread]",5539
50,"[ma100_spread, ts_slope20]",5462
69,"[ts_slope10, ma50_spread]",5450
44,"[ma50_spread, NormCloseChange_60]",5406
13,"[ma10_spread, ma50_spread]",5390


In [12]:
import backtesting.backtest_stats as bs
import pandas as pd

import shared.statistics as stats
pnl_frame = pd.concat(backtest_list)
pnl_frame = pnl_frame[pnl_frame['hourly_normalized_volume']<0.25]

bs.get_indicator_rr_double_table(trade_data=pnl_frame,indicator_list=['NormCloseChange_60','ma50_spread'],
                                 strategy_class='ts',
                                 long_pnl_field='NormPnl60',
                                            short_pnl_field='NormPnl60_')

,indicator1_ulimit,indicator2_ulimit,mean_pnl,reward_risk
0,-0.096218,-0.321776,69.974334,0.066256
1,-0.096218,-0.094570,-75.481946,-0.079913
2,-0.096218,NaN,-36.780351,-0.039411
3,0.100102,-0.076270,50.873909,0.059322
4,0.100102,0.088462,-59.529464,-0.073097
5,0.100102,NaN,-18.022800,-0.018619
6,NaN,0.103187,-35.263709,-0.035298
7,NaN,0.328586,-3.468466,-0.004051
8,NaN,NaN,101.779785,0.102270


In [5]:
# How Much Money Per Contract?

import pandas as pd

pnl_frame = pd.concat(backtest_list)
select_frame = pnl_frame[(pnl_frame['hourly_normalized_volume']<0.25)&(pnl_frame['tickerHead']=='C')]

long_frame = select_frame[(select_frame['ma50_spread']<-0.4)]
print('Long Pnl Per Contract: ' + str(long_frame['PerContractPnl60'].mean()))

short_frame = select_frame[(select_frame['ma50_spread']>0.4)]
print('Short Pnl Per Contract: ' + str(-short_frame['PerContractPnl60'].mean()))

print('Long Success Ratio: ' + str(100*sum(long_frame['PerContractPnl60']>=0)/sum(long_frame['PerContractPnl60'].notnull())))
print('Short Success Ratio: ' + str(100*sum(short_frame['PerContractPnl60']<=0)/sum(short_frame['PerContractPnl60'].notnull())))

print('Long Fail Ratio: ' + str(100*sum(long_frame['PerContractPnl60']<0)/sum(long_frame['PerContractPnl60'].notnull())))
print('Short Fail Ratio: ' + str(100*sum(short_frame['PerContractPnl60']>0)/sum(short_frame['PerContractPnl60'].notnull())))

long_winner = long_frame[long_frame['NormPnl60']>=0]
long_looser = long_frame[long_frame['NormPnl60']<0]

short_winner = short_frame[short_frame['NormPnl60']<=0]
short_looser = short_frame[short_frame['NormPnl60']>0]

print(long_winner['PerContractPnl60'].mean())
print(long_looser['PerContractPnl60'].mean())

print(-short_winner['PerContractPnl60'].mean())
print(-short_looser['PerContractPnl60'].mean())

Long Pnl Per Contract: -4.479166666666667
Short Pnl Per Contract: 0.628698224852071
Long Success Ratio: 56.1111111111
Short Success Ratio: 58.5798816568
Long Fail Ratio: 43.8888888889
Short Fail Ratio: 41.4201183432
34.40594059405941
-54.19303797468354
40.90909090909091
-56.339285714285715


In [28]:
import matplotlib.pyplot as plt
plt.plot(long_frame['PerContractPnl60'].cumsum())
plt.grid()                
plt.show()

In [34]:
pnl_frame = pd.concat(backtest_list)
pnl_frame = pnl_frame[pnl_frame['hourly_normalized_volume']<0.25]

long_frame = pnl_frame[(pnl_frame['ma50_spread']<-0.4)&(pnl_frame['tickerHead']=='B')]



long_frame.iloc[0]
long_frame.iloc[0]
selected_frame = long_frame[['ticker','settle_date','hour_minute','PerContractPnl60']]
selected_frame.iloc[1]



ticker                  BF2017
settle_date         2016-11-01
hour_minute               1235
PerContractPnl60           140
Name: 2016-11-01 12:35:00, dtype: object

In [35]:
import futures_charts.intraday_charts as ic
ic.get_intraday_breakout_chart(ticker='BF2017',trade_date=20161101)

In [34]:
import pandas as pd
import shared.statistics as stats
pnl_frame = pd.concat(backtest_list)
pnl_frame = pnl_frame[pnl_frame['hourly_normalized_volume']<0.25]

unique_settles = pnl_frame['settle_date'].unique()

indicator_name = 'ts_slope10'


value_list = stats.get_number_from_quantile(y = pnl_frame[indicator_name].values,quantile_list=[10,90])

long_pnl_list = []
short_pnl_list = []
num_trades_list = []

for i in range(len(unique_settles)):
    selected_frame = pnl_frame[pnl_frame['settle_date']==unique_settles[i]]
    big_frame = selected_frame[(selected_frame['ma50_spread']>0.4)]
    small_frame = selected_frame[(selected_frame['ma50_spread']<-0.4)]
    long_pnl_list.append(small_frame['NormPnl60'].sum())
    short_pnl_list.append(-big_frame['NormPnl60'].sum())
    num_trades_list.append(len(small_frame.index)+len(big_frame.index))
    

portfolio_frame = pd.DataFrame.from_items([('settle_date', unique_settles), 
                         ('long', long_pnl_list),
                         ('short', short_pnl_list),
                          ('num_trades',num_trades_list)])
portfolio_frame['total'] = portfolio_frame['long'] + portfolio_frame['short'] 

In [35]:
import matplotlib.pyplot as plt
plt.scatter(portfolio_frame['num_trades'],portfolio_frame['total'])
plt.grid()                
plt.show()

In [36]:
16*portfolio_frame['total'].mean()/portfolio_frame['total'].std()

3.5382787725833

In [37]:
import matplotlib.pyplot as plt
plt.plot(portfolio_frame['total'].cumsum())
plt.grid()                
plt.show()

In [5]:
# Which Tickerheads Are More Profitable?

ticker_head_list = pnl_frame['tickerHead'].unique()
import pandas as pd


pnl_frame = pd.concat(backtest_list)
pnl_frame = pnl_frame[pnl_frame['hourly_normalized_volume']<0.25]

long_pnl_list = []
short_pnl_list = []
long_rr_list = []
short_rr_list = []

for i in range(len(ticker_head_list)):
    selected_frame = pnl_frame[pnl_frame['tickerHead']==ticker_head_list[i]]
    big_frame = selected_frame[(selected_frame['ma50_spread']>0.4)]
    small_frame = selected_frame[(selected_frame['ma50_spread']<-0.4)]
    long_pnl_list.append(small_frame['PerContractPnl60'].mean())
    short_pnl_list.append(-big_frame['PerContractPnl60'].mean())
    
    long_rr_list.append(small_frame['NormPnl60'].mean()/small_frame['NormPnl60'].std())
    short_rr_list.append(-big_frame['NormPnl60'].mean()/big_frame['NormPnl60'].std())
                        
                        
pd.DataFrame.from_items([('tickerHead', ticker_head_list), 
                         ('long_total', long_pnl_list),
                         ('short_total', short_pnl_list),
                         ('long_rr', long_rr_list),
                         ('short_total', short_rr_list)])                        

,tickerHead,long_total,short_total,long_rr,short_total
0,C,-15.909091,-8.894231,-0.220672,0.122878
1,CC,198.723404,-16.515152,0.629273,0.189957
2,ES,-11.408730,-16.964286,-0.039275,0.123249
3,FV,-12.051197,25.007659,-0.213329,-0.360118
4,HO,74.781818,20.001639,0.269069,-0.054624
5,JY,21.562500,63.476562,0.102325,-0.369749
6,LN,12.627119,-37.840000,0.055590,0.212699
7,NG,149.941860,-6.371681,0.570414,0.024550
8,NQ,10.229885,71.837607,0.036047,-0.470631
9,RB,156.047500,-120.453435,0.377972,0.297987


In [9]:

import pandas as pd
pnl_frame = pd.concat(backtest_list)

selected_frame = pnl_frame[pnl_frame['trade_type']=='range_short']


import matplotlib.pyplot as plt
plt.plot(selected_frame['NormPnl60'].cumsum())
plt.grid()                
plt.show()


In [2]:
import pandas as pd
pnl_frame = pd.concat(backtest_list)

selected_frame = pnl_frame[pnl_frame['trade_type']=='ma_long']
print(selected_frame['NormPnl60'].mean()/selected_frame['NormPnl60'].std())

selected_frame = pnl_frame[pnl_frame['trade_type']=='ma_short']
print(selected_frame['NormPnl60'].mean()/selected_frame['NormPnl60'].std())

# ewma20-ewma100 just looking at level gives contrarian results
# ewma10-ewma50 just looking at level gives contrarian results
# ewma50-ewma100-lma240 just looking at level gives contrarian result 

# ewma50-lma240 crossover: contrarian result
# ewma10-ewma50 crossover: contrarian result
# ewma20-ewma100 crossover: weak momentum result

# close-lma240 crossover: weak momentum
# close-ewma50 crossover: no result
# close-ewma 100 crossover: no result



0.19788711353320365
0.07209830873790621


In [2]:
import pandas as pd
pnl_frame = pd.concat(backtest_list)


pnl_frame['NormPnl60PerContract'] = pnl_frame['NormPnl60']/(5000/(pnl_frame['contract_multiplier']*pnl_frame['daily_noise']))
selected_frame1 = pnl_frame[(pnl_frame['trade_type']=='trend_long')|(pnl_frame['trade_type']=='trend_short')]
selected_frame2 = pnl_frame[(pnl_frame['trade_type']=='range_long')|(pnl_frame['trade_type']=='range_short')]
selected_frame3 = pnl_frame[(pnl_frame['trade_type']=='ma_long')|(pnl_frame['trade_type']=='ma_long')]

selected_frame_list = [selected_frame1,selected_frame2,selected_frame3]

num_profitable_trades_list = []
average_pnl_list = []
average_pnl_contract_list = []
average_pnlws_list = []
average_pnlws_contract_list = []
average_win_contract_list = []
average_lose_contract_list = []
success_probability = []

for i in range(3):
    selected_frame = selected_frame_list[i]
    num_profitable_trades_list.append(sum(selected_frame['NormPnl60']>0))
    average_pnl_list.append(selected_frame['NormPnl60'].mean())
    average_pnl_contract_list.append(selected_frame['NormPnl60PerContract'].mean())
    
    winner_frame = selected_frame[selected_frame['NormPnl60PerContract']>=0]
    looser_frame = selected_frame[selected_frame['NormPnl60PerContract']<0]
    average_win_contract_list.append(winner_frame['NormPnl60PerContract'].mean())
    average_lose_contract_list.append(looser_frame['NormPnl60PerContract'].mean())
     
    average_pnlws_list.append(selected_frame['NormPnl60WS'].mean())
    average_pnlws_contract_list.append(selected_frame['NormPnl60WSPerContract'].mean())
    success_probability.append(100*sum(selected_frame['NormPnl60']>=0)/sum(selected_frame['NormPnl60'].notnull()))
    

pd.DataFrame.from_items([('numProfitableTrades', num_profitable_trades_list),
                         ('averagePnl',average_pnl_list),
                         ('averagePnlContract',average_pnl_contract_list),
                         ('averageWinContract',average_win_contract_list),
                         ('averageLoseContract',average_lose_contract_list),
                         ('averagePnlWS',average_pnlws_list),
                         ('averagePnlWSContract',average_pnlws_contract_list),
                         ('successProbability',success_probability)])   

,numProfitableTrades,averagePnl,averagePnlContract,averageWinContract,averageLoseContract,averagePnlWS,averagePnlWSContract,successProbability
0,1493,132.448073,14.494795,148.823699,-146.863705,41.726896,5.626193,54.570637
1,673,92.637461,12.182486,149.976383,-155.597647,11.033967,2.007312,54.906542
2,257,305.615263,41.853226,174.176952,-139.754337,49.019157,11.161677,57.849462


In [3]:
grouped  = selected_frame3.groupby('settle_date')

print('sharp ratio: '+ str(16*grouped['NormPnl60'].mean().mean()/grouped['NormPnl60'].mean().std()))
grouped['settle_date'].agg(['count'])
    

sharp ratio: 7.30674482866102


,count
settle_date,
2016-11-01,17
2016-11-02,7
2016-11-03,18
2016-11-04,6
2016-11-07,3
2016-11-08,12
2016-11-09,28
2016-11-10,8
2016-11-11,33


In [4]:
import matplotlib.pyplot as plt
plt.plot(grouped['NormPnl60'].mean().cumsum())

plt.grid()         
         
plt.show()


In [4]:
import pandas as pd
tickerhead_groups = selected_frame3.groupby('ticker_head')

tickerhead_stats = pd.DataFrame()

tickerhead_stats['total'] = tickerhead_groups['NormPnl60'].sum()
tickerhead_stats['rr'] = tickerhead_groups['NormPnl60'].mean()/tickerhead_groups['NormPnl60'].std()

tickerhead_stats



,total,rr
ticker_head,,
AD,12265.988723,0.557593
B,4919.723859,0.775837
BO,949.928152,0.092005
BP,1969.590196,0.114272
C,-4746.416781,-0.366852
CC,12282.211029,0.531457
CD,7172.582985,0.258387
CL,2418.908893,0.283527
CT,7999.288876,0.351879


In [5]:
import pandas as pd
pnl_frame = pd.concat(backtest_list)


selected_frame = pnl_frame[(pnl_frame['trade_type']=='ma_long')|(pnl_frame['trade_type']=='ma_short')]

winner_frame = selected_frame[selected_frame['NormPnl60WS']>0]
looser_frame = selected_frame[selected_frame['NormPnl60WS']<0]

print('Num Profitable Trades: ' + str(sum(selected_frame['NormPnl60WS']>0)))

print('Success Percentage: ' + str(100*sum(selected_frame['NormPnl60']>0)/len(selected_frame['NormPnl60'].notnull())))
print('Fail Percentage: ' + str(100*sum(selected_frame['NormPnl60']<0)/len(selected_frame['NormPnl60'].notnull())))



print('Average Winning Trade: ' + str(winner_frame['NormPnl60'].mean()))
print('Average Losing Trade: ' + str(looser_frame['NormPnl60'].mean()))
print('Average Trade: ' + str(selected_frame['NormPnl60'].mean()))
print('Average Trade Per Contract: ' + str(selected_frame['NormPnl60WSPerContract'].mean()))
print('Average Trade Without Stop: ' + str(selected_frame['NormPnl60'].mean()))

print('Average Winning Holding Period: ' + str(winner_frame['Holding_Period'].mean()))
print('Average Losing Holding Period: ' + str(looser_frame['Holding_Period'].mean()))


Num Profitable Trades: 377
Success Percentage: 45.1942740286
Fail Percentage: 36.4008179959
Average Winning Trade: 1195.798978538495
Average Losing Trade: -416.165854966891
Average Trade: 233.1564298903171
Average Trade Per Contract: 11.017929162357618
Average Trade Without Stop: 233.1564298903171
Average Winning Holding Period: 34.15119363395225
Average Losing Holding Period: 17.246376811594203


In [11]:
selected_frame = pnl_frame[(pnl_frame['trade_type']=='ma_long')|(pnl_frame['trade_type']=='ma_short')]



import matplotlib.pyplot as plt
plt.plot(grouped['NormPnl60'].sum().cumsum())

plt.grid()         
         
plt.show()


In [10]:
grouped = selected_frame.groupby('settle_date')
16*grouped['NormPnl60'].sum().mean()/grouped['NormPnl60'].sum().std()



4.64374750992965

In [3]:


import pandas as pd
pnl_frame = pd.concat(backtest_list)


selected_frame = pnl_frame[(pnl_frame['trade_type']=='ma_long')|(pnl_frame['trade_type']=='ma_short')]
#selected_frame = pnl_frame[pnl_frame['ticker_head']=='BP']

winner_frame = selected_frame[selected_frame['NormPnl15WSRobust']>0]
looser_frame = selected_frame[selected_frame['NormPnl15WSRobust']<0]

print('Num Profitable Trades: ' + str(sum(selected_frame['NormPnl15WSRobust']>0)))
print('Success Percentage: ' + str(100*sum(selected_frame['NormPnl15WSRobust']>0)/len(selected_frame['NormPnl15WSRobust'].notnull())))
print('Fail Percentage: ' + str(100*sum(selected_frame['NormPnl15WSRobust']<0)/len(selected_frame['NormPnl15WSRobust'].notnull())))

print('Average Winning Trade: ' + str(winner_frame['NormPnl15WSRobustPerContract'].mean()))
print('Average Losing Trade: ' + str(looser_frame['NormPnl15WSRobustPerContract'].mean()))
print('Average Trade: ' + str(selected_frame['NormPnl15WSRobust'].mean()))
print('Average Trade Hold 15 Minutes: ' + str(selected_frame['NormPnl15'].mean()))

print('Average Winning Holding Period: ' + str(winner_frame['Holding_Period'].mean()))
print('Average Losing Holding Period: ' + str(looser_frame['Holding_Period'].mean()))



KeyError: 'NormPnl15WSRobust'

In [26]:
import pandas as pd
import shared.calendar_utilities as cu
pnl_frame = pd.concat(backtest_list)
pnl_frame['trade_date'] = [x.to_datetime().date() for x in pnl_frame.index]
selected_frame = pnl_frame[(pnl_frame['trade_type']=='range_long')|(pnl_frame['trade_type']=='range_short')]
selected_frame.columns
wuhu = selected_frame[(selected_frame['trade_date']==cu.convert_doubledate_2datetime(20170119).date())&(selected_frame['ticker_head']=='BP')]

#cu.convert_doubledate_2datetime(20170119)
#selected_frame['trade_date']

wuhu['NormPnl15WSRobust'].mean()

75.0294379316596

In [29]:


import matplotlib.pyplot as plt
plt.plot(wuhu['NormPnl15WSRobust'].cumsum())

plt.grid()         
         
plt.show()


In [13]:
pnl_frame['D1_D2'] = pnl_frame['D1']-pnl_frame['D2']

selected_frame = pnl_frame[(pnl_frame['D1_D2']>=20)&(pnl_frame['D1_D2']<=30)]
#selected_frame = pnl_frame[(pnl_frame['D1_D2']>=0)&(pnl_frame['D1_D2']<=10)]
#selected_frame = pnl_frame[(pnl_frame['D1_D2']>0)]
print(selected_frame['NormPnl15WSRobust'].mean())
print(selected_frame['NormPnl15WSRobust'].mean()/selected_frame['NormPnl15WSRobust'].std())

91.63606561461852
0.1797900531957565


In [15]:
# Huge Discrepancy Between Longs and Shorts which tickerhead is the cause?
# this seems pretty widespread something must be wrong with the code

ticker_head_list = pnl_frame['ticker_head'].unique()
ticker_head_list
long_pnl_list = []
short_pnl_list = []

for i in range(len(ticker_head_list)):
    selected_frame = pnl_frame[pnl_frame['ticker_head']==ticker_head_list[i]]
    long_frame = selected_frame[selected_frame['D1']>selected_frame['D2']]
    short_frame = selected_frame[selected_frame['D1']<selected_frame['D2']]
    long_pnl_list.append(long_frame['Holding_Period'].mean())
    short_pnl_list.append(short_frame['Holding_Period'].mean())
    

pd.DataFrame.from_items([('tickerHead', ticker_head_list),
                         ('longPnl', long_pnl_list),
                         ('shortPnl', short_pnl_list)])
    
    

,tickerHead,longPnl,shortPnl
0,AD,7.198655,2.823269
1,B,6.393744,2.615901
2,BO,7.130919,2.766023
3,BP,6.659608,2.616846
4,C,9.226214,4.375502
5,CC,7.203343,2.996285
6,CD,6.594466,2.675422
7,CL,6.588352,2.662035
8,CT,6.593993,2.726155
9,EC,6.358923,2.650268


In [32]:
import matplotlib.pyplot as plt


selected_frame = pnl_frame[(pnl_frame['D1_D2']>0)]
plt.plot(selected_frame['NormPnl15WSRobust'].cumsum())

plt.grid()         
         
plt.show()



In [33]:

import matplotlib.pyplot as plt
import pandas as pd
pnl_frame = pd.concat(backtest_list)
selected_trades = pnl_frame
selected_trades = pnl_frame[pnl_frame['ticker_head']=='KW']
plt.plot(range(len(selected_trades.index)),selected_trades['NormPnl15WSRobust'].cumsum(),
         range(len(selected_trades.index)),selected_trades['NormPnl15'].cumsum())
plt.legend(['with stop','no stop'])
plt.grid()         
         
plt.show()

In [31]:
import pandas as pd
pnl_frame = pd.concat(backtest_list)
pnl_frame['pnlPerContract'] = pnl_frame['NormPnl15WSRobust']/pnl_frame['Qty']
ticker_head_list = pnl_frame['ticker_head'].unique()
ticker_head_list
i = 10
print(ticker_head_list[i])
pnl_frame[pnl_frame['ticker_head']==ticker_head_list[i]]['pnlPerContract'].mean()

# AD 1.8
# BO 2.5
# BP 2.7

ES


4.6875

In [32]:

import matplotlib.pyplot as plt
selected_frame = pnl_frame[pnl_frame['ticker_head']=='ES']

plt.plot(selected_frame['pnlPerContract'].cumsum())
plt.grid()         
         
plt.show()

In [17]:
import pandas as pd
pnl_frame = pd.concat(backtest_list)
pnl_frame = pnl_frame[(pnl_frame['trade_type']=='range_long')|(pnl_frame['trade_type']=='range_short')]

pnl_frame['trade_date'] = [x.to_datetime().date() for x in pnl_frame.index]

unique_dates = pnl_frame['trade_date'].unique()
unique_dates

daily_pnl_list = []

for i in range(len(unique_dates)):
    daily_pnl_list.append(pnl_frame[pnl_frame['trade_date']==unique_dates[i]]['NormPnl60WS'].sum())
    
    

In [12]:
daily_pnl_list

[-1731.726261663137,
 -2490.5071160018088,
 2966.5041393152874,
 6443.5751037920045,
 -1377.5513515227763,
 2439.150795511667,
 29471.750504156516,
 -13591.748552911182,
 -2414.8301389251237,
 -4640.453353853986,
 3472.1395958102826,
 -1147.8724283520123,
 3791.655777782865,
 2725.04643194705,
 15686.392436835315,
 1945.5572626735834,
 -771.001135592852,
 -1819.8098369339832,
 -4676.805106116678,
 1422.7478201198794,
 6524.466848877288,
 -3062.8132810814786,
 -878.3564516089447,
 12825.950855095427,
 -15947.772743431004,
 20358.55861762797,
 -387.27795890377405,
 -4853.706459028844,
 -8210.543297710254,
 -3560.5352436582734,
 0.13604057065049346,
 -110.66595204885726,
 722.3559316528122,
 -5162.246407309885,
 -1547.0584100775006,
 -2574.885730054787,
 1687.362254233149,
 -8270.724782453268,
 -168.76950114450722,
 13209.556361551282,
 1110.123521043275,
 463.3981191747826,
 7568.24138490626,
 265.32963423372126,
 -4442.451739137183,
 -1434.3115145543782,
 5689.771531244054,
 -24476.6122

In [13]:
import numpy as np
daily_pnl_array = np.array(daily_pnl_list)
16*np.mean(daily_pnl_array)/np.std(daily_pnl_array)

# range trades have a sharp ratio of 6.5; 130K PNL
# range trades using yesterdays high-low have a sharp ratio of 2.9; 100K PNL
# range trades without looking at morning range has a sharp ratio of 3.4; 160K pnl
# range trades except for strong trends have a sharp ratio of 4; 200K pnl
# range trades except for strong trens 
# 1 minute sharp: 13.3; 450K
# 1 minute with slower stochastic: 9.7; 280K
# 1 minute sharp with moving averages: 12.2; 450K
# 1 minute sharp with no moving averages: 16; 1M, mean pnl: $3

0.18168103763969851

In [5]:
import pandas as pd
pnl_frame = pd.concat(backtest_list)
pnl_frame['pnlPerContract'] = pnl_frame['NormPnl15WSRobust']/pnl_frame['Qty']
pnl_frame['pnlPerContract'].mean()

3.439958209593218

In [18]:
import numpy as np
import matplotlib.pyplot as plt
daily_pnl_array = np.array(daily_pnl_list)
np.sum(daily_pnl_array)/np.std(daily_pnl_array)

plt.plot(range(len(daily_pnl_array)),daily_pnl_array.cumsum())
plt.grid()         
         
plt.show()



In [4]:
import pandas as pd
pnl_frame = pd.concat(backtest_list)
unique_ticker_head_list = pnl_frame['ticker_head'].unique()
total_pnl_list = []
tlong_pnl_list = []
tshort_pnl_list = []
rlong_pnl_list = []
rshort_pnl_list = []

for i in range(len(unique_ticker_head_list)):
    selected_frame = pnl_frame[pnl_frame['ticker_head']==unique_ticker_head_list[i]]
    total_pnl_list.append(selected_frame['NormPnl60WSRobust'].sum())
    tlong_pnl_list.append(selected_frame[selected_frame['trade_type']=='ma_long']['NormPnl60WSRobust'].sum())
    tshort_pnl_list.append(selected_frame[selected_frame['trade_type']=='trend_short']['NormPnl60WSRobust'].sum())
    rlong_pnl_list.append(selected_frame[selected_frame['trade_type']=='range_long']['NormPnl60WSRobust'].sum())
    rshort_pnl_list.append(selected_frame[selected_frame['trade_type']=='range_short']['NormPnl60WSRobust'].sum())
    

summary_frame = pd.DataFrame.from_items([('ticker_head', unique_ticker_head_list), 
                         ('pnl', total_pnl_list),
                         ('trend_long', tlong_pnl_list),
                         ('trend_short', tshort_pnl_list),
                         ('range_long', rlong_pnl_list),
                         ('range_short', rshort_pnl_list)])
summary_frame

,ticker_head,pnl,trend_long,trend_short,range_long,range_short
0,AD,13968.595365,3086.277644,468.587054,5178.523305,5235.207363
1,B,-2034.349926,-3908.483426,443.471696,1661.910273,-231.248469
2,BP,10682.243630,1046.498559,-602.730513,8026.740887,2211.734696
3,CC,-1614.401647,-530.712588,-2517.966177,-2368.530218,3802.807336
4,CD,2057.348057,-1519.741915,-2772.320770,1550.420392,4798.990350
5,CL,-6205.358350,-5393.490940,-550.502557,2941.808710,-3203.173564
6,EC,7171.347448,-158.023631,-552.346984,6799.634742,1082.083320
7,ES,-634.640480,-3644.441225,1156.212634,-1038.858543,2892.446654
8,FC,-1207.216574,-1764.763258,-799.398530,182.246591,1174.698623
9,FV,41.567378,-5720.253976,-1495.683052,7202.562065,54.942341


In [8]:
print(summary_frame['pnl'].sum())
print(summary_frame['trend_long'].sum())
print(summary_frame['trend_short'].sum())
print(summary_frame['range_long'].sum())
print(summary_frame['range_short'].sum())

98681.29106757024
-28362.408606166937
-4357.856531156459
86195.78413327836
45205.77207161525


In [20]:
trend_long_frame = pnl_frame[pnl_frame['trade_type']=='trend_long']
trend_long_frame

trend_long_frame_sorted = trend_long_frame.sort('NormPnl60WSRobust')
trend_long_frame_sorted.iloc[6]

# Yellen Speech on 19th December
# CDH already close to highs after crossover
# BO trend might have changed by 1030
# C trend might have changed by 1235






open                                  18.635
high                                  18.695
low                                   18.635
close                                 18.695
K                                         50
D1                                  26.61228
D2                                  19.88001
D1_Lag                              12.57719
D2_Lag                              24.20787
CloseChange15                          -0.02
CloseChange60                          -0.12
high_Lag                              18.665
low_Lag                               18.555
long_stop                             18.555
short_stop                            18.695
long_stop_dist                         -0.14
short_stop_dist                        -0.14
hour_minute                             1055
bullish_stochastic_crossover            True
bearish_stochastic_crossover           False
e_bullish_stochastic_crossover          True
e_bearish_stochastic_crossover         False
m_bullish_

In [21]:
import futures_charts.intraday_charts as ic
ic.get_intraday_breakout_chart(ticker='SBH2017',trade_date=20161227,hour_minute_from=830)


In [9]:
import signals.intraday_technical_scalper as its
#signals_output = its.get_technical_scalper_4ticker(ticker='ECH2017',date_to=20170105)
trade_data = signals_output['trade_data']
trade_data.iloc[6]

open                                 1.063025
high                                 1.063025
low                                  1.062575
close                                1.062675
ma20                                 1.062842
K                                    28.81356
D1                                   44.63277
D2                                   50.65913
D1_Lag                               54.23729
D2_Lag                               54.87699
ma20_Lag                             1.062865
close_Lag                            1.063025
CloseChange15                        0.000125
CloseChange60                        0.000125
high_Lag                              1.06305
low_Lag                              1.062775
long_stop                            1.062575
short_stop                            1.06305
long_stop_dist                         0.0001
short_stop_dist                       -0.0001
hour_minute                              1230
bullish_stochastic_crossover      

In [9]:

hloc_data['ma20'] = pd.rolling_mean(hloc_data['close'], 20)
hloc_data['ma20']


2017-01-05 00:45:00         NaN
2017-01-05 00:50:00         NaN
2017-01-05 00:55:00         NaN
2017-01-05 01:00:00         NaN
2017-01-05 01:05:00         NaN
2017-01-05 01:10:00         NaN
2017-01-05 01:15:00         NaN
2017-01-05 01:20:00         NaN
2017-01-05 01:25:00         NaN
2017-01-05 01:30:00         NaN
2017-01-05 01:35:00         NaN
2017-01-05 01:40:00         NaN
2017-01-05 01:45:00         NaN
2017-01-05 01:50:00         NaN
2017-01-05 01:55:00         NaN
2017-01-05 02:00:00         NaN
2017-01-05 02:05:00         NaN
2017-01-05 02:10:00         NaN
2017-01-05 02:15:00         NaN
2017-01-05 02:20:00    1.058729
2017-01-05 02:25:00    1.058555
2017-01-05 02:30:00    1.058364
2017-01-05 02:35:00    1.058159
2017-01-05 02:40:00    1.057908
2017-01-05 02:45:00    1.057629
2017-01-05 02:50:00    1.057390
2017-01-05 02:55:00    1.057181
2017-01-05 03:00:00    1.057000
2017-01-05 03:05:00    1.056865
2017-01-05 03:10:00    1.056735
                         ...   
2017-01-

In [13]:
import get_price.get_futures_price as gfp
ticker = 'ECH2017'
daily_settles = gfp.get_futures_price_preloaded(ticker=ticker)
daily_settles.iloc[-1]

settle_date        2017-01-05 00:00:00
ticker                         ECH2017
ticker_head                         EC
ticker_month                         3
ticker_year                       2017
cal_dte                             67
tr_dte                              45
open_price                      1.0516
high_price                      1.0646
low_price                       1.0511
close_price                     1.0622
volume                          281224
cont_indx                       201703
change1                            NaN
change1_instant                    NaN
change2                            NaN
change5                            NaN
change10                           NaN
change20                           NaN
change_5                        0.0164
change_1                       0.01235
Name: 35874, dtype: object

In [2]:
import futures_charts.intraday_charts as ic
ic.get_intraday_futures_spread_chart(ticker_list=['SH2017','BOH2017','SMH2017'],date_to=20170109)